In [6]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

### Read in the required datasets

In [90]:
#### DEMEANED ####
## counts
fnl_large_core_counts_dm = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_all_core_dm/final_main_large_cities_core_counts_dm.csv')
fnl_medium_core_counts_dm = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_all_core_dm/final_main_medium_cities_core_counts_dm.csv')
fnl_small_core_counts_dm = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_all_core_dm/final_main_small_cities_core_counts_dm.csv')

## rates
fnl_large_core_rates_dm = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_all_core_dm/final_main_large_cities_core_rates_dm.csv')
fnl_medium_core_rates_dm = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_all_core_dm/final_main_medium_cities_core_rates_dm.csv')
fnl_small_core_rates_dm = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_all_core_dm/final_main_small_cities_core_rates_dm.csv')

#### create ist rep pop value between 2000-2003 column 

In [148]:
def create_pop_wt_col(req_dfs):
    for df in req_dfs:
        df_wt = df.query('YEAR >= 2000 and YEAR <= 2010').loc[:, ['ORI', 'YEAR', 'fnl_population']]
        df_wt['pop_wt'] = df_wt.groupby('ORI')['fnl_population'].head(1)
        
        df_wt.dropna(subset = ['pop_wt'], inplace=True)
        
        df = df.merge(df_wt, on='ORI', how='left')
        df.rename({'YEAR_x': 'YEAR', 'fnl_population_x': 'fnl_population'}, axis=1, inplace=True)
        df.drop(['YEAR_y', 'fnl_population_y'], axis=1, inplace=True)
        #print(df.loc[:, ['ORI', 'YEAR', 'fnl_population', 'pop_wt']].head(50))
        #print(df.query('ORI == "AL00102"').loc[:, ['ORI', 'YEAR', 'fnl_population', 'pop_wt']])
        print(df.loc[df['pop_wt'].isnull().shape[0]])

In [149]:
create_pop_wt_col([fnl_large_core_counts_dm])

KeyError: 9120

In [ ]:
def create_lag_vars(req_dfs, vars_to_lag, groupby_var, num_of_lags):
    for df in req_dfs:
        count=1
        for lag_num in range(num_of_lags):
            for var in vars_to_lag:
                df[f'lag{count}_{var}'] = df.groupby(f'{groupby_var}')[f'{var}'].shift(count)
            count+=1

#### create lags for counts

In [47]:
create_lag_vars(req_dfs=[fnl_large_core_counts_dm, fnl_medium_core_counts_dm, fnl_small_core_counts_dm], 
                vars_to_lag=['dm_violent_crime', 'dm_total_officers', 
                             'dm_prison_occupancy_count', 'dm_jail_occupancy_count', 
                             'dm_drug_tot_arrests', 'dm_disorder_arrests_tot_index'],
                groupby_var = 'ORI',
               num_of_lags=2)

#### create lags for rates

In [48]:
create_lag_vars(req_dfs=[fnl_large_core_rates_dm, fnl_medium_core_rates_dm, fnl_small_core_rates_dm], 
                vars_to_lag=['dm_violent_crime_rate', 'dm_total_officers_rate', 
                             'dm_prison_occupancy_count_rate', 'dm_jail_occupancy_count_rate',
                             'dm_drug_tot_arrests_rate', 'dm_disorder_arrests_tot_index_rate'],
                groupby_var = 'ORI',
               num_of_lags=2)

In [55]:
print(fnl_large_core_rates_dm.loc[:, ['lag2_dm_violent_crime_rate', 'lag2_dm_total_officers_rate']].head(50))

    lag2_dm_violent_crime_rate  lag2_dm_total_officers_rate
0                          NaN                          NaN
1                          NaN                          NaN
2                  -217.777470                    -0.163062
3                   -78.897099                    -0.122272
4                    40.067695                    -0.138435
5                   130.250108                    -0.103398
6                   227.467281                    -0.194396
7                   226.209430                     0.008953
8                    36.907314                     0.060783
9                   -50.094830                     0.083217
10                 -161.592000                     0.132665
11                 -150.246170                     0.071809
12                 -219.493489                     0.035592
13                 -141.858723                    -0.044121
14                 -159.837496                    -0.058179
15                 -158.859834          

In [ ]:
count_df.pop_lag = count_df.groupby('COWname')['SP.POP.TOTL'].shift(2)